In [32]:
import os
import json

import scrapegraphai

In [2]:
# This is a workaround for the issue with Jupyter notebooks and asyncio
import nest_asyncio
nest_asyncio.apply()

In [3]:
import dotenv

dotenv.find_dotenv()

''

In [ ]:
graph_config = {
    "provider": "openai",
    "output": "python",
    "llm": {
        "model": "gpt-3.5-turbo",
        "api_key": os.getenv("OPENAI_API_KEY"),
    },
    "library": "BeautifulSoup",
}

In [28]:
graph_config = {
    "output": "python",
    "provider": "ollama",
    "llm": {
        "model": "ollama/mistral",
        "temperature": 0,
        "format": "json",  # Ollama needs the format to be specified explicitly
        "base_url": "http://localhost:11434",  # set Ollama URL
    },
    "embeddings": {
        "model": "ollama/nomic-embed-text",
        "base_url": "http://localhost:11434",  # set Ollama URL
    },
    "library": "BeautifulSoup",
}

In [42]:
# Define the prompt, source, and configuration
prompt = "Iterate through each tournament in the source and extract the following fields: id, url, name, date, country, region, and number of players. Return the data as a list of dictionaries."
source = "https://braacket.com/league/nemelee/tournament?rows=20"

# Create the script creator graph
script_creator = scrapegraphai.graphs.ScriptCreatorMultiGraph(prompt, source, graph_config)

result = script_creator.run()
print(result)

{}


In [43]:
script_json = json.loads(script_creator.final_state['scripts'][0])
script_str = '\n'.join(script_json['main'])
print(script_str)

url = 'SOURCE_URL'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
tournaments = []
tournament_tags = soup.find_all('TAG_FOR_TOURNAMENT')
for tournament_tag in tournament_tags:
data = {
'id': tournament_tag['ID_ATTRIBUTE'],
'url': tournament_tag['URL_ATTRIBUTE'],
'name': tournament_tag.find('NAME_TAG').text.strip(),
'date': tournament_tag.find('DATE_TAG').text.strip(),
'country': tournament_tag.find('COUNTRY_TAG').text.strip(),
'region': tournament_tag.find('REGION_TAG').text.strip(),
'number_of_players': int(tournament_tag.find('NUMBER_OF_PLAYERS_TAG').text.strip())
}
tournaments.append(data)
return json.dumps(tournaments, indent=4)
}


In [44]:
# Define the prompt, source, and configuration
prompt = "Iterate through each tournament in the source. For each tournament go to the tournament page. Then go to the mateches detail. In the matches detail page go to each stage. On a stage page extract the following fields: winner_id, winner, loser_id, loser, winner_score, loser_score, stage_id, stage_name, tournament_id. Return the data as a list of dictionaries."
source = "https://braacket.com/league/nemelee/tournament?rows=200"

# Create the script creator graph
script_creator = scrapegraphai.graphs.ScriptCreatorMultiGraph(prompt, source, graph_config)

result = script_creator.run()
print(result)

{"script": [
      {
         "url": "https://example.com/tournament",
         "method": "GET",
         "data": {},
         "response_type": "json"
      },
      {
         "url": "{tournament_url}/match",
         "method": "GET",
         "params": {"tournament_url": "{tournament['url']}"},
         "data": {},
         "response_type": "json"
      },
      {
         "url": "{match_url}/stage",
         "method": "GET",
         "params": {"match_url": "{match['url']}"},
         "data": {},
         "response_type": "json"
      },
      {
         "script": [
            "const data = []",
            "for (let tournament of tournaments)",
            "  for (let match of tournament.matches)",
            "    for (let stage of match.stages)",
            "      const winner_id = stage.winner_id",
            "      const loser_id = stage.loser_id",
            "      const winner = stage.winner",
            "      const loser = stage.loser",
            "      const winner_

In [45]:
script_json = json.loads(script_creator.final_state['scripts'][0])
script_str = '\n'.join(script_json['main'])
print(script_str)

url = 'SOURCE_URL'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')
tournaments = soup.find_all('selector_for_tournament')
data = []
for tournament in tournaments:
tournament_url = tournament['attribute_with_url']
tournament_soup = BeautifulSoup(requests.get(tournament_url).content, 'html.parser')
matches = tournament_soup.find_all('selector_for_match')
for match in matches:
match_url = match['attribute_with_url']
match_soup = BeautifulSoup(requests.get(match_url).content, 'html.parser')
stages = match_soup.find_all('selector_for_stage')
for stage in stages:
winner_id = stage['attribute_with_winner_id']
winner = stage['attribute_with_winner']
loser_id = stage['attribute_with_loser_id']
loser = stage['attribute_with_loser']
winner_score = stage['attribute_with_winner_score']
loser_score = stage['attribute_with_loser_score']
stage_id = stage['attribute_with_stage_id']
stage_name = stage['attribute_with_stage_name']
tournament_id = stage['attribute_with_tournament_id']
data.